In [14]:
import os
import glob
import time

import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

import warnings

# Functions

In [15]:
ERA5M_PATH  = "/global/homes/w/wboos/m3310project/wboos/era5monthlyQuentin/"
ERA5M_PATH2 = "/global/project/projectdirs/m3310/wboos/era5monthly/"

boxNH = [[-100, -70, 12 , 20],
         [-80 , -60, 8  , 16],
         [70  , 90 , 2  , 18],
         [100 , 120, 8  , 20],
         [118 , 140, 4  , 20],
         [100 , 110, 0  , 12],
        ]

boxSH = [[20  , 50 , -20, 0 ],
         [-60 , -30, -20, 0 ],
        ]

boxNH1 = [[b[0]%360,b[1]%360,b[2],b[3]] for b in boxNH]
boxSH1 = [[b[0]%360,b[1]%360,b[2],b[3]] for b in boxSH]

namesNH = ["Central America",
           "Colombia/Venezuela",
           "South Asia",
           "Vietnam",
           "Philippines",
           "Malaysia"
          ]
namesSH = ["Tanzania",
           "Brazil"
          ]
plots_order=[0,1,2,3,7,6,5,4]

In [16]:
#Data extraction from ERA5
era5yrs = list(range(1998,2014))

def era5_file(year,varid):
    """return file path to an ERA5 variable for the year 'year'
    varid gives the id of the variable in era5
        - year : str, "YYYY"
        - varid : str, eg. "128_130_t" for temperature
    """
    try :
        fpath = glob.glob(os.path.join(ERA5M_PATH,"*/e5.*.%s.*.%s*.nc"%(varid,year)))[0]
    except IndexError :
        path = "/global/cscratch1/sd/qnicolas/era5monthly/"
        fpath = glob.glob(os.path.join(path,"*/e5.*.%s.*.%s*.nc"%(varid,year)))[0]
    return fpath


def climatology_era5(varid):
    paths = [era5_file(year,varid) for year in era5yrs]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore",FutureWarning)
        ds = xr.open_mfdataset(paths).groupby("time.month").mean("time").compute()
        varname = list(ds.data_vars)[0] #get name of the main variable, eg 'T' for temperature
        return ds[varname]
    
def climatology_era5_level(month,varid,level):
    paths = [era5_file(year,varid) for year in era5yrs]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore",FutureWarning)
        ds = xr.open_mfdataset(paths).sel(level=level).groupby("time.month").mean("time").compute()
        varname = list(ds.data_vars)[0] #get name of the main variable, eg 'T' for temperature
        return ds[varname]


def region_mean2D(variable,mask,box):
    """Given a 2D variable (lat, lon), compute a spatial mean within a specified region
    defined by a mask, inside a given box
        - variable = 3D xarray.dataarray. Dimensions must be named "latitude" and "longitude"
        - mask = 2D xarray.dataarray of 0s and 1s. Must have same grid and dimension names as 'variable'
        - box = list of four items, [lon1, lon2, lat1, lat2]
    """
    mask_box = mask.fillna(0).sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2]))
    variable_box = variable.fillna(0).sel(longitude=slice(box[0],box[1]),latitude=slice(box[3],box[2]))
    maskedvar = variable_box*mask_box
    region_mean = maskedvar.fillna(0).sum(["latitude","longitude"])/ mask_box.sum(["latitude","longitude"])
    return region_mean

def region_mean2D_trmm(variable,mask,box):
    mask_box = mask.fillna(0).sel(longitude=slice(box[0],box[1]),latitude=slice(box[2],box[3]))
    variable_box = variable.fillna(0).sel(longitude=slice(box[0],box[1]),latitude=slice(box[2],box[3]))
    maskedvar = variable_box*mask_box
    region_mean = maskedvar.fillna(0).sum(["latitude","longitude"])/ mask_box.sum(["latitude","longitude"])
    return region_mean


    return vartot/len(era5yrs)



# Masks, data

In [17]:
def slide_coord(mask):
    """Switch the longitude coord from (-180,180) to (0,360) 
    and change coordinate names from 'LAT1','LON1' to 'latitude' and 'longitude'"""
    mask0=mask.copy()
    mask0.coords['longitude'] = mask0.coords['LON1']%360
    mask1 = mask0.swap_dims({'LON1': 'longitude'}).rename({'LAT1': 'latitude'})

    #Sort the longitude values
    sort_inds = {"longitude": np.argsort(mask1["longitude"].values)}
    mask1 = mask1.isel(**sort_inds)
    return mask1

masks = xr.open_dataset("/global/cscratch1/sd/qnicolas/autumnMonsoonData/winter_rainfall_masks.nc")
##/!\ temporary change for access on Spin
##masks = xr.open_dataset("winter_rainfall_masks.nc")
trmm_nh_winter_mask = masks.TRMM_NH_WINTER_MASK.fillna(0.)
trmm_sh_winter_mask = masks.TRMM_SH_WINTER_MASK.fillna(0.)


## To be modified
REFERENCE_GRID = xr.open_dataset(ERA5M_PATH+"e5.moda.an.pl/e5.moda.an.pl.128_060_pv.ll025sc.1979010100_1979120100.nc").PV.sel(latitude=slice(50., -50.)).isel(time=0) #to get the era5 grid

trmm_nh_winter_mask1 = (slide_coord(trmm_nh_winter_mask).interp_like(REFERENCE_GRID) > 0)*1.
trmm_sh_winter_mask1 = (slide_coord(trmm_sh_winter_mask).interp_like(REFERENCE_GRID) > 0)*1.
trmm_nh_winter_mask = slide_coord(trmm_nh_winter_mask)
trmm_sh_winter_mask = slide_coord(trmm_sh_winter_mask)                     

In [19]:
t=time.time()
cape_seasmean = climatology_era5('128_059_cape')
print(time.time()-t)

5.024801969528198


# CAPE values

In [27]:
from tabulate import tabulate

print(' '*20 + '\033[1m'+ "  CAPE values in J/kg (ERA5 mean 1998-2013)" + '\033[0m')

tab=[]

for i,box in enumerate(boxNH1):
    cape_autumn_mean = region_mean2D(cape_seasmean.sel(month=slice(10,12)).mean('month'), trmm_nh_winter_mask1, box)
    cape_spring_mean = region_mean2D(cape_seasmean.sel(month=slice(4,6)).mean('month'), trmm_nh_winter_mask1, box)
    tab.append([namesNH[i], "%.2f"%cape_autumn_mean, "%.2f"%cape_spring_mean])

for i,box in enumerate(boxSH1):
    cape_autumn_mean = region_mean2D(cape_seasmean.sel(month=slice(4,6)).mean('month'), trmm_sh_winter_mask1, box)
    cape_spring_mean = region_mean2D(cape_seasmean.sel(month=slice(10,12)).mean('month'), trmm_sh_winter_mask1, box)
    tab.append([namesSH[i], "%.2f"%cape_autumn_mean, "%.2f"%cape_spring_mean])


print(tabulate(tab, headers=['Region'+' '*17, 'Autumn (AMJ (SH)/ OND(NH)), ','Spring (OND (SH)/ AMJ(NH))']))


                      CAPE values in J/kg (ERA5 mean 1998-2013)
Region                       Autumn (AMJ (SH)/ OND(NH)),     Spring (OND (SH)/ AMJ(NH))
-------------------------  ------------------------------  ----------------------------
Central America                                    719.01                        987.31
Colombia/Venezuela                                 819.35                        814.42
South Asia                                         473.2                        1389.05
Vietnam                                            423.24                       1234.39
Philippines                                        632.63                        865.12
Malaysia                                           588.61                       1158.02
Tanzania                                           399.25                        371.59
Brazil                                             315.5                         252.31


In [29]:
from tabulate import tabulate

print(' '*20 + '\033[1m'+ "  CAPE values in J/kg (ERA5 mean 1998-2013)" + '\033[0m')

tab=[]

for i,box in enumerate(boxNH1):
    cape_autumn_mean = region_mean2D(cape_seasmean.sel(month=11), trmm_nh_winter_mask1, box)
    cape_spring_mean = region_mean2D(cape_seasmean.sel(month=5), trmm_nh_winter_mask1, box)
    tab.append([namesNH[i], "%.2f"%cape_autumn_mean, "%.2f"%cape_spring_mean])

for i,box in enumerate(boxSH1):
    cape_autumn_mean = region_mean2D(cape_seasmean.sel(month=5), trmm_sh_winter_mask1, box)
    cape_spring_mean = region_mean2D(cape_seasmean.sel(month=11), trmm_sh_winter_mask1, box)
    tab.append([namesSH[i], "%.2f"%cape_autumn_mean, "%.2f"%cape_spring_mean])


print(tabulate(tab, headers=['Region'+' '*17, 'Autumn (May (SH)/ November(NH)), ','Spring (November (SH)/ May(NH))']))


                      CAPE values in J/kg (ERA5 mean 1998-2013)
Region                       Autumn (May (SH)/ November(NH)),     Spring (November (SH)/ May(NH))
-------------------------  -----------------------------------  ---------------------------------
Central America                                         631.1                              941.84
Colombia/Venezuela                                      781.28                             814.26
South Asia                                              426.11                            1636.71
Vietnam                                                 452.75                            1328.16
Philippines                                             688.73                             952.72
Malaysia                                                597.8                             1254.99
Tanzania                                                371.74                             394.61
Brazil                                                